# Assignment: K-Nearest Neighbor
## Krishu Wadhwa - euj7fh
## October 09, 2024

Do two questions.

Questions Chosen: **Q0, Q1, and Q3**.

# Question 0

**1. What is the difference between regression and classification?**  
<u>Regression:</u> using data/features/covariates to predict a numeric outcome.  
<u>Classification:</u> using data/features/covariates to predict a categorical outcome.

**2. What is a confusion table? What does it help us understand about a model's performance?**  
A confusion matrix cross-tabulates actual and predicted values, showing both correct and incorrect predictions. It helps evaluate the model's performance by highlighting not just accuracy but also where it is making prediction errors.

**3. Define accuracy. Can an accurate model be flawed for practical use? Explain.**  
Accuracy is the proportion of cases where the model correctly predicts the outcome. However, an accurate model can still be flawed for practical use if it makes errors with significant consequences. For example, a model that predicts loan approvals might be accurate overall, but if it incorrectly approves risky loans, it could lead to financial losses. This shows that even with high accuracy, models may not be flawed, and therefore, not ideal for practical use.

**4. What does the SSE quantify about a particular model?**  
The sum of squared error (SSE) helps answer the question: "How far are the predictions from the true outcomes on the validation/test set?" It quantifies the total difference between actual values and the values predicted by a model (quantifies square error of the model). To compute SSE, for each observation in the validation set, you subtract the predicted value from the actual value, square the result, and then sum these squared differences across all observations. A lower SSE indicates a better fit, meaning the model’s predictions are closer to the actual values. Thus, SSE effectively measures how far off the predictions are from the actual outcomes.

**5. What are overfitting and underfitting?**  
<u>Overfitting:</u> occurs when the model is too complex to reliably explain the phenomenon of interest. This may capture additional unnecessary or irrelevant patterns in the training data, which makes it less reliable for new data.  
<u>Underfitting:</u> occurs when the model is too simple to reliably explain the phenomenon of interest. This may fail to capture important features or patterns in the data.   
Both overfitting and underfitting lead to instances of unreliable models. 

**6. Why does splitting the data into training and testing sets, and choosing `k` by evaluating accuracy or SSE on the test set, improve model performance?**  
Splitting the data into training and testing sets helps the model learn from the training data and then be evaluated on testing/unseen data. This simulates how the model would perform in real-world situations. By selecting `k` based on accuracy or SSE on the test set, we can better ensure that the model works well with new data. This reduces the chances of overfitting or underfitting the training set. Testing the model on testing/unseen data replicates how the model will behave in practical scenarios, providing a more accurate estimate of its performance. This also improves how reliable the model is.

**7. With classification, we can report a class label as a prediction or a probability distribution over class labels. Please explain the strengths and weaknesses of each approach.**  
When a model uses class labels, it provides a single, definitive outcome. The strength of this approach is that it is simple and easy to interpret. However, a weakness is that it doesn’t explain the model’s confidence in the prediction. For example, a model may predict Class A, but without probabilities, we don’t know if the confidence was 99% or 51%. In contrast, using a probability distribution over class labels shows the likelihood of each possible outcome. While a strength of this is that it provides a more detailed view of the model’s uncertainty, a weakness is that it can be harder to interpret and may require additional steps to make a final classification. For example, decision-making is harder when probabilities are close, such as 51% vs. 49%.

# Question 1  
This question is a case study for `k` nearest neighbor regression, using the `USA_cars_datasets.csv` data.  
The target variable `y` is `price` and the features are `year` and `mileage`.

**1. Load the `./data/USA_cars_datasets.csv`. Keep the following variables and drop the rest: `price`, `year`, `mileage`. Are there any `NA`'s to handle? Look at the head and dimensions of the data.**

**2. Maxmin normalize year and mileage.**

**3. Split the sample into ~80% for training and ~20% for evaluation.**

**4. Use the `k`nn algorithm and the training data to predict `price` using `year` and `mileage` for the test set for `k = 3, 10, 25, 50, 100, 300`. For each value of `k`, compute the mean squared error and print a scatterplot showing the test value plotted against the predicted value. What patterns do you notice as you increase `k`?**

**5. Determine the optimal `k` for these data.**

**6. Describe what happened in the plots of predicted versus actual prices as `k` varied, taking your answer into part 6 into account. (Hint: Use the words "underfitting" and "overfitting".)**

# Question 3
This question is a case study for `k` nearest neighbor regression, using the `cars_env.csv` data.  
The variables that go into the model are one more dimension to think about: We can use the train-test split approach to pick the variables that go into the model, not just the number of neighbors.

**1. Load the data. We're going to use `footprint`, `baseline mpg`, `baseline price`, and `baseline sales`. Prepare some EDA results for these variables: describe tables, histograms/kernel density plots, scatterplots, etc. I renamed these variables to `footprint`, `mpg`, `price`, and `sales` to save time.**

**2. Maxmin normalize footprint, mpg, and price. These will be our features/covariates `X`. Our target/dependent/outcome variable `y` will be sales. Does it make sense to normalize `y`?**

**3. Make a 30% train-test split of the data into 30% test/validation data and 70% training data.**

**4. Using all three covariates --- `footprint`, `mpg`, and `price` --- what's the best `k` to use? What SSE is achieved on the test set? To answer these questions, evalute the sum of squared error on the test set for a reasonable range of values of `k` (perhaps 2 to 150), and find the `k` with the lowest SSE.**

**5. Do part 4 again, for each pair of variables: `footprint` and `mpg`, `footprint` and `price`, `mpg` and `price`.**

**6. Which set of variables from parts 4 and 5 does the best, in terms of minimizing SSE at the optimal choice of neighbors? Explain.**